In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [1]:
from torchvision import datasets, transforms

# Define data transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Load training dataset
train_dataset = datasets.ImageFolder(root='<path to training data>', transform=transform)

In [2]:
from torchvision import models

# Load pre-trained ResNet50 model
model = models.resnet50(pretrained=True)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 127MB/s]


In [3]:
import torch.nn as nn

# Modify final classifier layer for binary classification
in_features = model.fc.in_features
model.fc = nn.Linear(in_features, 2)  # 2 output classes (resume and non-resume)

In [4]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [5]:
from torch.utils.data import DataLoader

# DataLoader for training set
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
num_epochs = 10

# Training loop
for epoch in range(num_epochs):
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms

from torch.utils.data import DataLoader
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_fscore_support

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])


validation_dataset = datasets.ImageFolder(root='<path to testing data>', transform=transform)
validation_loader = DataLoader(validation_dataset, batch_size=2, shuffle=False)

model.eval()
all_preds = []
all_labels = []
with torch.no_grad():
    for inputs, labels in validation_loader:
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Calculate accuracy
accuracy = (torch.tensor(all_preds) == torch.tensor(all_labels)).float().mean()

# Calculate confusion matrix
conf_matrix = confusion_matrix(all_labels, all_preds)

# Calculate precision, recall and f1 score
precision, recall, f1_score, _ = precision_recall_fscore_support(all_labels, all_preds, average='binary')

print(f"Epoch {epoch + 1}/{num_epochs} - Accuracy: {accuracy.item():.4f}")
print("Confusion Matrix:")
print(conf_matrix)
print(f"Precision: {precision:.4f}")
print(f"Recall (Sensitivity): {recall:.4f}")
print(f"F1 Score: {f1_score:.4f}")